# Variational autoencoders

In this project, we will explore the idea of variational autoencoders. Also, we will implement a variational autoencoder using Tensorflow to generate images like MNIST hand-written digits.

## Table of Contents
- [1 - Introduction](#1)
- [2 - Autoencoders](#2)
    - [2.1 - Dimension Reduction using Autoencoders](#2.1)
- [3 - Variational autoencoders](#3)
    - [3.1 - Theory behind VAEs](#3.1)
        - [3.1.1 - Evidence Lower BOund](#3.1.1)
        - [3.1.2 - Variational Inference](#3.1.2)
        - [3.1.3 - Reparameterization Trick](#3.1.3)
        - 
        - [3.1.5 - Hand-written Image Generator](#3.1.5)

<a name='1'></a>
## Introduction
Variational autoencoders are members of deep generative models that encode the inputs to a lower dimensional space, sample randomly from the latent or encoded space, and decode that sample to generate a new data. More percisely, variational autoencoders can be considered in two different ways:
1. VAEs are type of autoencoders. However, there is a key difference between AEs and VAEs that will be discussed when autoencoders are explained.
2. VAEs are probabilistic generative models that work with joint distribution of observed and latent variables

<a name='2'></a>
## Autoencoders

<a name='2.1'></a>
### Dimension Reduction using Autoencoders

<a name='3'></a>
## Variational autoencoders

<a name='3.1'></a>
### Theory behind VAEs

<a name='3.1.1'></a>
#### Evidence Lower BOund
Evidence lower bound (ELBO) is a key element in order to approximate posterior distribution. It is a lower bound on the log marginal likelihood of the data given the model parameters. By maximizing the ELBO, we are effectively minimizing the Kullback-Leibler (KL) divergence between the true posterior distribution and the approximating distribution(i will show it in this section).

Before finding this lower bound, it is a good idea to define our problem(i'll define our problem and our notation for each part). Consider we have two random variables, $X$(observation or real data) and $Z$(hidden or latent variable). $X$ and $Z$ are distributed according to a joint distribution $P(X,Z;\theta)$ where $\theta$ parameterizes the distribution.

Now we need to determine what is the evidence. The evidence is simply a log-likelihood of observations $x$ with fixed $\theta$. Intuitively, likelihood function shows how much our model and parameter($\theta$) align with observations. High value of likelihood function indicates that the model is appropriate for the given data. So, now the goal is to find a lower bound for $p(x;\theta)$. Assume that $Z$ follows $q$ distribution. Now we can use marginalization to achieve that lower bound as follows:

$$\begin{align} log p(x;\theta) &= log\int_{z} p(x,z;\theta) dz \\ &= log\int_{z} p(x,z;\theta) \frac{q(z)}{q(z)} dz \\ &= log E_{z\sim q}[\frac{p(x,Z;\theta)}{q(Z)}] \\ & \geq^* E_{z\sim q}[log \frac{p(x,Z;\theta)}{q(Z)}]\end{align}$$

$$\begin{align} => ELBO = E_{z\sim q}[log \frac{p(x,Z;\theta)}{q(Z)}] \ (1) \end{align}$$

\* : This inequality is the result of [Jensen's inequality](https://en.wikipedia.org/wiki/Jensen%27s_inequality). Since $log$ is a concave function, $log(E[X]) \geq E[log(X)]$ where X in this statement is $\frac{p(x,Z;\theta)}{q(Z)}$.

Now lets prove why maximizing the ELBO leads to minimizing the KL divergence between the true posterior distribution and the approximating distribution. Assume that we want to find a $q$ distribution that is the most accurate distribution in order to approximate $p(z|x;\theta)$.(in VAEs we need to find an approximation for $p(z|x;\theta)$, so it is needed to find a close distribution for that and it is the reason why i explain the statement with this distribution).

$$\begin{align} KL \ (q(z) \ || \ p(z|x;\theta)) &= E_{Z \sim q}[log \frac{q(Z)}{p(Z|x;\theta)}] \\ &= E_{Z \sim q}[log \ q(Z)] - E_{Z \sim q}[log \ p(Z|x;\theta)] \\ &= E_{Z \sim q}[log \ q(Z)] - E_{Z \sim q}[log \frac {p(Z,x;\theta)}{p(x;\theta)}] \\ &= E_{Z \sim q}[log \ q(Z)] - E_{Z \sim q}[log \ p(Z,x;\theta)] + E_{Z \sim q}[log \ p(x;\theta)] \\ &= E_{Z \sim q}[log \ p(x;\theta)] - E_{Z \sim q}[log\frac{p(x,Z;\theta)}{q(Z)}] \\ &=^{*,(1)} log \ p(x;\theta) - ELBO \\ &= evidence - ELBO\end{align}$$

\* : Notice that $log \ p(x;\theta)$ is not dependent on $Z$, so it acts like a constant and comes out of the expectation.

Now, notice that when $\theta$ is fixed and we are looking for a q that minimize the KL divergence, evidence acts as constant, so becuase of negative sign of ELBO, maximizing the ELBO leads to minimizing the KL divergence.

<a name='3.1.2'></a>
#### Variational Inference

In many practical scenarios, computing the exact posterior distribution is intractable due to the complexity of the model or the size of the data. Variational inference provides a framework to approximate this posterior distribution with a simpler distribution chosen from a parameterized family of distributions. 
The essence of variational inference is to pose the problem of approximating the posterior distribution as an optimization problem. The goal is to find the member of the chosen family of distributions that minimizes the Kullback-Leibler (KL) divergence from the true posterior distribution.
By maximizing the Evidence Lower Bound (ELBO), which is a lower bound on the log marginal likelihood of the data given the model parameters, variational inference seeks to find the best approximation to the true posterior distribution given the model and the observed data.

When we have a model with both hidden (Z) and observed (X) variables, and we want to figure out the likelihood of Z given X, variational inference helps ous to find a reasonable approximation. But, first lets check why we can not calculate $p(z|x)$ explicitly. Using Bayes theorem, $p(z|x)$ can be written as follows:
$$\begin{align} p(z|x) = \frac{p(z,x)}{p(x)} = \frac{p(x|z) \times p(z)}{\int_z p(z,x) dz} \end{align}$$

The important point is that calculating the denominator is not always feasible. Therefore, variational inference is used.

In this technique, we consider a family of distributions named variational distribution family and aim to find a $q$ that $q(z)$ be the closest distribution to $p(z|x)$ in the variational distribution family. Also, assume that $\phi$ controls these distributions.(it is the variational parameter). So, our goal is to find a $\phi$ that minimize the KL divergence between $q_{\phi}(z)$ and $p(z|x)$. Interestingly, based on what we showed in [Evidence Lower BOund](#3.1.1), instead of minimizing the KL divergence, we can maximize the ELBO to find the best approximation. Consequently, now we have to deal with $\phi$ becuase it is a parameter than characterize the family of distributions.
$$\begin{align} ELBO(\phi) &= L(\phi) = E_{Z \sim q_{\phi}}[log \ p(x,Z) - log \ q(Z)] \\ \hat{\phi} &= argmax_{\phi} \ L(\phi) \end{align}$$

<a name='3.1.3'></a>
#### Reparameterization Trick

In a typical variational inference setting, we aim to approximate the posterior distribution over the latent variables using a variational distribution. When the variational distribution involves sampling from a distribution like a Gaussian, it introduces randomness into the computation of gradients during the optimization process.
The reparameterization trick helps to address this issue by reparameterizing the random variable in such a way that the stochasticity is separated from the parameters of the distribution. This allows gradient-based optimization algorithms to flow through the random sampling process, leading to lower variance gradient estimates and more stable training. So, this technique enables ous to perform stochastic gradient ascent on the ELBO. Now it is obvious how we will approximate the posterior distribution, we will use optimization strategies to find the best parameters in our case.

##### **High Variance Gradients**
First lets explore what happens if we do not use reparameterization trick. Let’s say we want to take the gradient w.r.t. $\theta$ of the $E_{x \sim p}[f_{\theta}(x)]$. If the p distribution is not parameterized by $\theta$, then the gradient can easily go inside of the expectation. However, if it is not the case, then we can not calculate the gradient in general.

$$\begin{align} \nabla_{\theta} \ E_{x \sim p_{\theta}}[f_{\theta}(x)] &= \nabla_{\theta} \int_{x} p_{\theta}(x) f_{\theta}(x) dx \\ &= \int_{x}\nabla_{\theta} ( p_{\theta}(x) f_{\theta}(x)) dx \\ &= \int_{x}\nabla_{\theta}p_{\theta}(x) f_{\theta}(x)dx + \int_{x} p_{\theta}(x) \nabla_{\theta} f_{\theta}(x) dx \\ &= \int_{x}\nabla_{\theta}p_{\theta}(x) f_{\theta}(x)dx + E_{x \sim p_{\theta}}[\nabla_{\theta}f_{\theta}(x)]\end{align} $$

The first term in the final expression is not guaranteed to be an expectation. While Monte Carlo methods necessitate the ability to sample from $p(x)$, they do not inherently require access to its gradient information. This limitation becomes challenging when dealing with scenarios where computing the gradient of $p(x)$ with respect to $\theta$ analytically is infeasible, which is often the case in practice. Therefore, in order to use Monte Carlo methods, we need to apply changes that remove $\nabla_{\theta} \ p_{\theta}(x)$ from our integrals.

##### **Reparameterization Trick Explanation**
Sometimes the random variable $x \sim p_{\theta}(x)$ can be reparameterized as a deterministic function $g$ of $\theta$ and of a random variable $\epsilon \sim D$, where $D$ is not parameterized on $\theta$.
$$\begin{align} \epsilon &\sim D \\ x &= g_{\theta}(\epsilon) \end{align}$$

Now lets see what happen if we take gradient from $E_{x \sim p}[f_{\theta}(x)]$:

$$\begin{align} \nabla_{\theta} \ E_{x \sim p_{\theta}}[f_{\theta}(x)] &= \nabla_{\theta} \ E_{\epsilon \sim g_{\theta}}[f(g_{\theta}(\epsilon))] \\ &=^* E_{\epsilon \sim g_{\theta}}[\nabla_{\theta} f(g_{\theta}(\epsilon))]\end{align} $$

\* : Notice that $p(\epsilon)$ or more generally $D$ are not dependent to $\theta$, so we can move the gradient inside the expectation.

As we showed, now we can use Monte Carlo method to estimate this expectation.

One may ask that how how the $D$ distribution can be found? Generally it is not a simple problem, but some family of distributions conduct to simple answers. For example, in our case we are dealing with Guassian distributions($q(z|x)$) so we can choose standard normal distribution and our $g$ function as: $g_{\theta}(\epsilon) = \mu + \sigma \epsilon$.

In order to use reparameterization trick we should consider these requirenments as well:
- $f_{\theta}(x)$ must be differentiable w.r.t x its input.
- $g_{\theta}(\epsilon)$ must exist and be differentiable w.r.t. $\theta$.

##### **Stochastic Gradient Ascent of the ELBO**
For this part we will apply following idea on our original problem which was discussed in [Evidence Lower BOund](#3.1.1) and [Variational Inference](#3.1.2), so all notations and symbols are with respect to these parts.

The goal of using variational inference is to find the closest $q$ to $p(z|x)$ using continuous optimization techniques like gradient ascent. Consequently, at each step, parameters are updated in this way:

$$\begin{align} \phi_{t+1} = \phi_t + \alpha \ \nabla_{\phi} \ L(\phi) \end{align}$$
Notic that we are trying to maximize the ELBO, so gradient ascent is used.

As we saw before, ELBO consists of an expectation, so the gradients face randomness in back propagation. In order to get rid of this randomness, we can use reparametrization trick. Before exploring this idea, we need to check another problem and it is how can gradients be calculated? As we can see, what makes our computation complex is dealing with integrals. Therefore, we use stochastic gradient descent that consider a distribution for gradients and at each step, sample a gradient from that distribution to update the $\phi$. Instead of computing the exact gradient of the ELBO with respect to $\phi$, we formulate a random variable $V(\phi)$ that holds $E[V(\phi)] = \nabla_{\phi} L(\phi)$.(it is just a formal way of expressing the SGD). 
$$\begin{align} v &\sim V(\phi) \\ \phi_{t+1} &= \phi_{t} + \alpha \ v \end{align}$$

First, we will reparameterize $q_{\phi}(z)$ in the following way:
$$\begin{align} \epsilon &\sim N(0,1) \\ z &\sim g_{\phi}(\epsilon) \end{align}$$

Where $g$ has the form that we talked in previous section.

Now we replace every $z$ in ELBO equation with $g_{\phi}$:
$$\begin{align} L(\phi) = E_{\epsilon \sim N(0,1)}[log \ p(x,g_{\phi}(\epsilon)) - log \ q_{\phi}(g_{\phi}(\epsilon))]\end{align}$$

As we showed previously, this reparameterization enables us to approximate the ELBO via Monte Carlo sampling. So we will sample L times from standard normal distribution, denote ith sample as $\epsilon_i$ and estimate the expectation as:
$$\hat{L(\phi)} = \frac{1}{L} \sum_{l=0}^{L}[log \ p(x,g_{\phi}(\epsilon_l)) - log \ q_{\phi}(g_{\phi}(\epsilon_l))]$$

Because of all requirenments are satisfied, we can take gradient from both sides:
$$\nabla_{\phi} \ \hat{L(\phi)} = \nabla_{\phi} \frac{1}{L} \sum_{l=0}^{L}[log \ p(x,g_{\phi}(\epsilon_l)) - log \ q_{\phi}(g_{\phi}(\epsilon_l))]$$
It is easy to show that $E[\nabla_{\phi} \ \hat{L(\phi)}] = \nabla_{\phi} \ L(\phi)$.(just needs to bring the $\nabla_{\phi}$ out of expectation and move the expectation inside the sigma). Therefore, $E[\nabla_{\phi} \ \hat{L(\phi)}]$ is what we were looking for.
$$V(\phi)= \nabla_{\phi} \ \hat{L(\phi)}$$
In simpler terms,  sampling $\epsilon_1,\dots,\epsilon_L$ from $N(0,1)$, computing an approximate ELBO, and finding the gradient to this approximation is basically like drawing random gradients from a distribution $V(\phi)$ where the average gradient matches the ELBO gradient. This is why it's important for $D$ to be easy to sample from when using the reparametrization trick: we rely on samples from $D$ to create samples for $V(\phi)$.

#### The probability model perspective
Now we have all neccessary tools to show full theory behind the VAEs. In this section we will talk about probabilistic aspect of VAEs. 

Inputs or samples are denoted as x and latent variables are denoted as z. The generative process is as follows:

1.First sample a latent variable z from latent space. $z \sim p_{\theta}(z)$.

2.Then map(using $f_{\theta}$) z to the parameters of another distribution to sample x. $x \sim p_{\theta}(x|z)$.

Variational autoencoders are a generative model in that they describe a joint distribution over samples and their associated latent variable, $p(x,z)$.

Although, we should consider the training process as well. So assume we are given a dataset consisting of data points $x_1,\dots,x_n$. As what i showed and what will be showed, in order to minimize our loss(KL divergence) we need two components:

1.For fixed $\theta$, for each x_i, $p_{\theta}(z_i|x_i)$ be computed.

2.MLE of $\theta$.

But we can not calculate these two without approximation because in order to compute these two, we need to marginalize out the $z$ in $p(x,z)$ and this means that an integral over all latent space is required which is intracitble.(I mentioned the problem more percisely in [variational inference](#3.1.2)). So in order to find an approximation solutions to these problems, variational inference is used. First, like previous parts, assume that $\theta$ is fixed. To approximate $p_{\theta}(z_i|x_i)$, we can use [variational inference](#3.1.2) and 
[evidence lower bound](#3.1.1). VI performs approximating by choosing a family of distributions and then finds the closest distribution to $p_{\theta}(z_i|x_i)$. 

Before defining our variational family, lets call the function that calculate the mean and standard deviation of x, $h_{\phi}$.($\phi$ is the parameter of the h function.) Note that in variational autoencoders this h is a neural network.
$$\begin{align} \mu &= h_{\phi}^1(x) \\ \sigma &= h_{\phi}^2(x) \end{align}$$

$$variational \ family = \{q_{\phi} | q_{\phi} \ \sim N(h_{\phi}^1(x), h_{\phi}^2(x))\}$$


In order to have an loss function to measure how much these two distributions are close to each other, we use Kullback–Leibler divergence between $q_{\phi}(z_i|x_i) \in variational \ family$(approximation distribution) and $p_{\theta}(z_i|x_i)$. Now we know that minimizing the KL divergence is equivalent to maximizing the ELBO.
$$\begin{align} L(\phi) &= E_{Z_1,\dots,Z_n \sim q_{\phi}}\sum_{i=1}^n[log \ p_{\theta}(x_i,z_i) - log \ q_{\phi}(z_i|x_i)]\end{align} \\ \sum_{i=1}^nE_{Z_i \sim q_{\phi}}[log \ p_{\theta}(x_i,z_i) - log \ q_{\phi}(z_i|x_i)]$$

So far we have assumed that $\theta$ is fixed and we did not optimize the ELBO w.r.t $\theta$. But $\theta$ is the parameter of the decoder function and we have to optimize decoder as well as encoder. Therefore, we can extend the definition of our loss function and consider ELBO as function of both $\phi$ and $\theta$. Now to goal is to find both $\phi$ and $\theta$ that maximize the ELBO. We can show that why it is true in the following way: we know $\theta$ is a parameter for both of $log \ p(x)$ and ELBO, so